In [1]:
import os
from google.oauth2 import service_account  # type: ignore
import googleapiclient.discovery  # type: ignore
from google.cloud import bigquery
import pandas as pd

In [3]:
# client = bigquery.Client()

# query = '''
# WITH transfers AS (
# SELECT 
#   block_timestamp, 
#   from_address, 
#   to_address, 
#   token_address, 
#   value  
# FROM `bigquery-public-data.crypto_ethereum.token_transfers`  
# WHERE TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-07-22"))
# SELECT
#   t.block_timestamp,
#   t.from_address,
#   t.to_address,
#   t.value,
#   tok.name,
#   tok.symbol,
#   tok.decimals,
#   tok.total_supply
# FROM transfers t
# LEFT JOIN `bigquery-public-data.crypto_ethereum.tokens` tok ON t.token_address = tok.address
# '''

# query_job = client.query(query)

# df = query_job.to_dataframe()
# df.to_csv('data/transactions.csv')
# df.head()

Para ahorrar capacidad computacional, he descargado la query en un csv y lo llamaré a este para optimizar el tiempo de análisis

## Voy a remover la columna total_supply porque tiene valores extraños y no confio en ellos para continuar trabajando con ellos (parecen no estar actualizados).
Una buena opción sería conseguir los datos de la oferta total de los tokens, o al menos tener un booleano que permita indicar cuales tienen capped supply. Esto sería útil para medir qué proporción de la oferta total de cada token se está transaccionando en esos momentos.

In [4]:
df = pd.read_csv('data/transactions.csv')
df = df[df.columns[1:-1]]
df['decimals'] = pd.to_numeric(df['decimals'], errors='coerce')
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df['value'] = df['value']/ (10**df['decimals'])
df['block_timestamp'] = pd.to_datetime(df['block_timestamp'])
df['tx_month'] = df['block_timestamp'].dt.month
df['tx_year'] = df['block_timestamp'].dt.year
df.head()

/var/folders/k4/6hfj9vfx0rvbk2skj_ydz9600000gn/T/ipykernel_99888/4137274742.py:1: DtypeWarning: Columns (4,5,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/transactions.csv')


,block_timestamp,from_address,to_address,value,name,symbol,decimals,tx_month,tx_year
0,2024-07-22 00:00:11+00:00,0x06fd4ba7973a0d39a91734bbc35bc2bcaa99e3b0,0x28c6c06298d514db089934071355e5743bf21d60,3.000000e+05,COTI Token,COTI,18.0,7,2024
1,2024-07-22 00:00:11+00:00,0x0000000000000000000000000000000000000000,0x42df00ac2e0000005486ac375a00b80000d12758,2.879000e+03,Metronome,MET,18.0,7,2024
2,2024-07-22 00:00:11+00:00,0x79320fad5780ec2109b2242b20f120ee16b5dafb,0x28c6c06298d514db089934071355e5743bf21d60,3.900000e+07,SHIBA INU,SHIB,18.0,7,2024
3,2024-07-22 00:00:11+00:00,0xa9c61fe59b5702b1d382fd1d5e495887ff34c21d,0x28c6c06298d514db089934071355e5743bf21d60,4.660659e+03,ChainLink Token,LINK,18.0,7,2024
4,2024-07-22 00:00:23+00:00,0x8670b29f42089019747b24f8d6965afa3eaf90ec,0x60d6649f0b58445d39d98aaf29d5de590111f67c,2.500000e+06,SHIBA INU,SHIB,18.0,7,2024


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1398518 entries, 0 to 1398517
Data columns (total 9 columns):
 #   Column           Non-Null Count    Dtype              
---  ------           --------------    -----              
 0   block_timestamp  1398518 non-null  datetime64[ns, UTC]
 1   from_address     1398518 non-null  object             
 2   to_address       1398518 non-null  object             
 3   value            543980 non-null   float64            
 4   name             542692 non-null   object             
 5   symbol           542692 non-null   object             
 6   decimals         543980 non-null   float64            
 7   tx_month         1398518 non-null  int32              
 8   tx_year          1398518 non-null  int32              
dtypes: datetime64[ns, UTC](1), float64(2), int32(2), object(4)
memory usage: 85.4+ MB


In [6]:
df.describe()

,value,decimals,tx_month,tx_year
count,5.439800e+05,543980.000000,1398518.0,1398518.0
mean,3.397146e+50,14.399787,7.0,2024.0
std,1.771701e+53,5.472407,0.0,0.0
min,0.000000e+00,0.000000,7.0,2024.0
25%,5.000000e-02,6.000000,7.0,2024.0
50%,3.446683e-01,18.000000,7.0,2024.0
75%,1.096815e+02,18.000000,7.0,2024.0
max,9.239898e+55,18.000000,7.0,2024.0


## Indicadores que tengo que elaborar:
- transacciones enviadas por esa address
- transacciones recibidas
- cantidad de criptos que contienen
- Activos enviados por unidad de tiempo (en cantidad y en variedad)
- Tiempo ocurrido entre transacciones
  

25/07 -> me dejo planteado variables para sumar al modelo

In [7]:
df_from_tx_count = pd.DataFrame(df.groupby(['from_address', 'tx_year', 'tx_month']).size())
df_from_tx_count.head()

,,,0
from_address,tx_year,tx_month,
0x0000000000000000000000000000000000000000,2024,7,83119
0x0000000000000000000000000000000000000001,2024,7,4
0x0000000000000000000000000000000000029e2c,2024,7,1
0x0000000000000000000000000000000000029f3f,2024,7,1
0x000000000000003607fce1ac9e043a86675c5c2f,2024,7,3


In [8]:
df_to_tx_count = pd.DataFrame(df.groupby(['to_address', 'tx_year', 'tx_month']).size())
df_from_tx_count.head()

,,,0
from_address,tx_year,tx_month,
0x0000000000000000000000000000000000000000,2024,7,83119
0x0000000000000000000000000000000000000001,2024,7,4
0x0000000000000000000000000000000000029e2c,2024,7,1
0x0000000000000000000000000000000000029f3f,2024,7,1
0x000000000000003607fce1ac9e043a86675c5c2f,2024,7,3


In [9]:
df.groupby(['from_address', 'symbol'])['value'].sum()

from_address                                symbol
0x0000000000000000000000000000000000000000  APT         1000.000000
                                            BST          188.938288
                                            COTI      276579.008140
                                            EKPE           0.000000
                                            FUSE      178200.000000
                                                          ...      
0xfffa78c979c2f787b16eac7c7e9c77b11feb77fb  WETH           0.005990
0xfffac831dee6b31d332566b563cee2ea5f1fd177  USDT         400.000000
0xfffd768351a42bd486f5ca8caf0c731eacdc19c2  USDT        1017.700000
0xfffd86577bf4eaa07bcd42ed792c7108342fe401  USDT           7.465713
0xfffef93628e1dca980af5f7a721a784ab3af501b  USDT       53013.000000
Name: value, Length: 75888, dtype: float64

In [10]:
df.groupby(['to_address', 'symbol'])['value'].sum()

to_address                                  symbol
0x0000000000000000000000000000000000000000  BOMB      4.200000e+01
                                            BURN      2.327162e+06
                                            CTP       3.067685e+10
                                            DAO       4.518331e+01
                                            FUSE      2.963697e+04
                                                          ...     
0xfffac831dee6b31d332566b563cee2ea5f1fd177  USDT      4.000000e+02
0xfffd768351a42bd486f5ca8caf0c731eacdc19c2  USDT      1.017700e+03
0xfffe0ba9c71a9d2fc8fdea29668b5ca9b23496ab  USDT      1.890000e+02
0xffff9fcf60cd147f204d57cdc89905e9b1ff7fd6  USDT      1.149642e+01
                                            WETH      2.491406e-02
Name: value, Length: 87340, dtype: float64